# Create a pycistopic object from count matrix
**Authorship:** Adam Klie (last updated: 08/21/2023)<br>
***
**Description:** This notebook is meant to build a pycistopic object from a count matrix.
***

# Set-up

In [1]:
# Import the necessary packages
import os
import sys
import glob
import pickle
import pandas as pd
from scipy import io
from pycisTopic.cistopic_class import *

In [24]:
# Params
mtx_file = "/cellar/users/aklie/data/igvf/beta_cell_networks/platinum/igvf_sc-islet_10X-Multiome/17Aug23/scATAC/matrix.mtx.gz"
barcode_file  = "/cellar/users/aklie/data/igvf/beta_cell_networks/platinum/igvf_sc-islet_10X-Multiome/17Aug23/scATAC/metadata.csv"
peaks_file = "/cellar/users/aklie/data/igvf/beta_cell_networks/platinum/igvf_sc-islet_10X-Multiome/17Aug23/scATAC/features.tsv.gz"
blacklist_file = "/cellar/users/aklie/data/igvf/references/blacklists/hg38/ENCFF356LFX.bed"
output_dir = "/cellar/users/aklie/projects/igvf/beta_cell_networks/infer_grns/scenicplus/results/igvf_sc-islet_10X-Multiome/17Aug23"

In [25]:
# Make output dir if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get data

In [26]:
# Read in the counts and send to csr
counts = io.mmread(mtx_file)
counts_csr = counts.tocsr()
counts_csr.shape

(262611, 83289)

In [26]:
# Read in the cell metadata
cell_data = pd.read_csv(barcode_file, index_col=0, low_memory=False)
cell_data.head()

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,nCount_RNA_raw,nFeature_RNA_raw,nCount_ATAC,nFeature_ATAC,nCount_SCT,nFeature_SCT,...,atac_pct.mt,TSS.enrichment,TSS.percentile,mpeak.weight,wsnn_res.0.3,cell.type.1,cell.type.2,predicted.cell.type,predicted.cell.type.score,barcode
MO1_AAACAGCCAGCAATAA-1,SeuratProject,7140.390126,2937,0.000000,7781,2945,10302,4964,2807,1654,...,0.629753,4.811552,0.15,0.150786,4,pre.SC.endocrine,pre.SC.alpha,pre.SC.alpha,1.00000,AAACAGCCAGCAATAA-1
MO1_AAACCAACAACCGCCA-1,SeuratProject,5083.686880,2651,0.000000,5564,2655,28678,11595,3028,2044,...,0.597243,5.251150,0.51,0.322520,11,SC.endocrine,SC.delta,SC.delta,1.00000,AAACCAACAACCGCCA-1
MO1_AAACCGCGTATTGTGG-1,SeuratProject,8302.654374,3797,0.000000,9338,3810,14258,6820,2693,1793,...,0.733843,4.083190,0.01,0.320034,4,pre.SC.endocrine,pre.SC.alpha,pre.SC.alpha,0.99152,AAACCGCGTATTGTGG-1
MO1_AAACGCGCAAGCCACT-1,SeuratProject,5772.429015,3107,0.005388,6430,3114,27699,11662,3011,2105,...,0.445260,4.810653,0.15,0.215760,3,SC.endocrine,SC.EC,SC.EC,1.00000,AAACGCGCAAGCCACT-1
MO1_AAACGCGCAGTTATCG-1,SeuratProject,2732.242968,1822,0.140443,2912,1824,19568,8592,2803,1816,...,1.289934,4.997553,0.27,0.357753,0,SC.endocrine,SC.beta,SC.beta,1.00000,AAACGCGCAGTTATCG-1


In [27]:
# Read in the regions, clean up and check how many
region_names = pd.read_csv(peaks_file, sep="\t", header=None)[0].values
region_names = [region.replace("-", ":", 1) for region in region_names]
len(region_names)

262611

# Make the object

In [29]:
# Create the object
cistopic_obj = create_cistopic_object(
    fragment_matrix=counts_csr,
    cell_names=cell_data.index,
    region_names=region_names,
    path_to_blacklist=blacklist_file,
    split_pattern="_",
    tag_cells=False,
    project="igvf_sc-islet_10X-Multiome_10Aug23"
)

2023-08-21 15:49:44,267 cisTopic     INFO     Removing blacklisted regions
2023-08-21 15:49:49,435 cisTopic     INFO     Creating CistopicObject


/cellar/users/aklie/opt/miniconda3/envs/scenicplus/lib/python3.8/site-packages/pycisTopic/cistopic_class.py:607: RuntimeWarning: divide by zero encountered in log10
  np.log10(cisTopic_nr_frag),
/cellar/users/aklie/opt/miniconda3/envs/scenicplus/lib/python3.8/site-packages/pycisTopic/cistopic_class.py:609: RuntimeWarning: divide by zero encountered in log10
  np.log10(cisTopic_nr_acc),


2023-08-21 15:50:12,932 cisTopic     INFO     Done!


In [ ]:
# Add cell metadata
cistopic_obj.add_cell_data(cell_data)

In [34]:
# Check the object
cistopic_obj.project, cistopic_obj.fragment_matrix.shape, cistopic_obj.cell_data.head(), cistopic_obj.region_names[:5]

('igvf_sc-islet_10X-Multiome_10Aug23',
 (262418, 83289),
                        cisTopic_nr_frag cisTopic_log_nr_frag cisTopic_nr_acc  \
 MO1_AAACAGCCAGCAATAA-1             6161             3.789651            5581   
 MO1_AAACCAACAACCGCCA-1            17040              4.23147           13284   
 MO1_AAACCGCGTATTGTGG-1             8642             3.936614            7781   
 MO1_AAACGCGCAAGCCACT-1            16741             4.223781           13457   
 MO1_AAACGCGCAGTTATCG-1            11428              4.05797            9567   
 
                        cisTopic_log_nr_acc  \
 MO1_AAACAGCCAGCAATAA-1            3.746712   
 MO1_AAACCAACAACCGCCA-1            4.123329   
 MO1_AAACCGCGTATTGTGG-1            3.891035   
 MO1_AAACGCGCAAGCCACT-1            4.128948   
 MO1_AAACGCGCAGTTATCG-1            3.980776   
 
                                                  sample_id     orig.ident  \
 MO1_AAACAGCCAGCAATAA-1  igvf_sc-islet_10X-Multiome_10Aug23  SeuratProject   
 MO1_AAACCAACAA

In [35]:
# Save the cistopic object
pickle.dump(
    cistopic_obj,
    open(os.path.join(output_dir, "cistopic_obj.pkl"), 'wb')
)

# Check the load

In [36]:
cistopic_obj_load = pickle.load(open(os.path.join(output_dir, "cistopic_obj.pkl"), 'rb'))

In [37]:
# Check the object
cistopic_obj_load.project, cistopic_obj_load.fragment_matrix.shape, cistopic_obj_load.cell_data.head(), cistopic_obj_load.region_names[:5]

('igvf_sc-islet_10X-Multiome_10Aug23',
 (262418, 83289),
                        cisTopic_nr_frag cisTopic_log_nr_frag cisTopic_nr_acc  \
 MO1_AAACAGCCAGCAATAA-1             6161             3.789651            5581   
 MO1_AAACCAACAACCGCCA-1            17040              4.23147           13284   
 MO1_AAACCGCGTATTGTGG-1             8642             3.936614            7781   
 MO1_AAACGCGCAAGCCACT-1            16741             4.223781           13457   
 MO1_AAACGCGCAGTTATCG-1            11428              4.05797            9567   
 
                        cisTopic_log_nr_acc  \
 MO1_AAACAGCCAGCAATAA-1            3.746712   
 MO1_AAACCAACAACCGCCA-1            4.123329   
 MO1_AAACCGCGTATTGTGG-1            3.891035   
 MO1_AAACGCGCAAGCCACT-1            4.128948   
 MO1_AAACGCGCAGTTATCG-1            3.980776   
 
                                                  sample_id     orig.ident  \
 MO1_AAACAGCCAGCAATAA-1  igvf_sc-islet_10X-Multiome_10Aug23  SeuratProject   
 MO1_AAACCAACAA

# DONE!

---

# Scratch

In [51]:
n_topics = ['2,', '4,', '8,', '16,', '32,', '48,', '64,', '80']

In [55]:
type(list(map(lambda x: int(x.split(',')[0]), n_topics))[0])

int

In [ ]:
list(map(int, n_topics.split(',')[0]))

In [46]:
n_topics = ["2", "4", "8", "16", "32", "48", "64", "80"]

In [47]:
n_topics = list(map(int, n_topics))

In [48]:
n_topics

[2, 4, 8, 16, 32, 48, 64, 80]

In [40]:
n_topics = list(map(int, n_topics[0].split(',')))

[2, 4, 8, 16, 32, 48, 64, 80]

In [57]:
from pycisTopic.lda_models import *

In [58]:
run_cgs_models_mallet?

Signature:
run_cgs_models_mallet(
    path_to_mallet_binary: str,
    cistopic_obj: 'cisTopicObject',
    n_topics: List[int],
    n_cpu: Union[int, NoneType] = 1,
    n_iter: Union[int, NoneType] = 150,
    random_state: Union[int, NoneType] = 555,
    alpha: Union[float, NoneType] = 50,
    alpha_by_topic: Union[bool, NoneType] = True,
    eta: Union[float, NoneType] = 0.1,
    eta_by_topic: Union[bool, NoneType] = False,
    top_topics_coh: Union[int, NoneType] = 5,
    tmp_path: Union[str, NoneType] = None,
    save_path: Union[str, NoneType] = None,
    reuse_corpus: Union[bool, NoneType] = False,
)
Docstring:
Run Latent Dirichlet Allocation per model as implemented in Mallet (McCallum, 2002).

Parameters
----------
path_to_mallet_binary: str
    Path to the mallet binary (e.g. /xxx/Mallet/bin/mallet).
cistopic_obj: cisTopicObject
    A :class:`cisTopicObject`. Note that cells/regions have to be filtered before running any LDA model.
n_topics: list of int
    A list containing the

In [4]:
import ray
ray.__version__

'2.1.0'

In [5]:
ray.init(num_cpus=4)

2023-08-21 19:00:10,940	INFO worker.py:1519 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.15
Ray version:,2.1.0
Dashboard:,http://127.0.0.1:8265


In [6]:
ray.shutdown()